# RAG Agent - Interactive Demo

This notebook lets you manually test every component of the RAG agent:

1. **Config** - loading from env vars or `config.yaml`
2. **Knowledge Base** - ingesting documents and running similarity search
3. **RAG Agent** - asking questions with retrieval-augmented generation

## Setup

Make sure you have either:
- A `.env` file with `OPENAI_API_KEY=sk-...` in the project root, **or**
- A `config.yaml` file (copy from `config.yaml.example`)

In [ ]:
# Add project root to path so we can import the agent package
import sys, os
sys.path.insert(0, os.path.abspath(".."))
os.chdir(os.path.abspath(".."))

## 1. Configuration

The `Config` class loads settings with this priority:  
**env vars > config.yaml > defaults**

In [ ]:
from agent.config import Config

# Option A: Auto-detect (loads .env + config.yaml if they exist)
cfg = Config()
print(cfg)

# Option B: Env-only mode (ignores config.yaml)
# cfg = Config(env_only=True)

# Option C: Explicit YAML path
# cfg = Config(config_path="path/to/my_config.yaml")

# Validate that required settings are present
cfg.validate()
print("Config is valid.")

## 2. Knowledge Base

The knowledge base uses ChromaDB for vector storage and OpenAI embeddings.

### 2a. Initialize

In [ ]:
from openai import OpenAI
from agent.knowledge_base import KnowledgeBase

client = OpenAI(api_key=cfg.openai_api_key)

# Use persist_directory=None for in-memory (no files written to disk)
kb = KnowledgeBase(
    openai_client=client,
    embedding_model=cfg.embedding_model,
    persist_directory=None,  # in-memory for notebook testing
)
print(f"Knowledge base initialized. Documents stored: {kb.document_count}")

### 2b. Load documents from the sample directory

In [ ]:
n_chunks = kb.load_directory(cfg.knowledge_base_path)
print(f"Loaded {n_chunks} chunks from '{cfg.knowledge_base_path}'")
print(f"Total documents in KB: {kb.document_count}")

### 2c. Add custom text on the fly

In [ ]:
custom_text = """
The company's return policy allows customers to return items within 30 days
of purchase. Items must be in original packaging and unused condition.
Refunds are processed within 5-7 business days after the return is received.
"""

kb.load_text(custom_text, source="inline-return-policy")
print(f"Total documents in KB: {kb.document_count}")

### 2d. Test similarity search directly

In [ ]:
results = kb.query("What is the return policy?", n_results=3)

for i, r in enumerate(results, 1):
    print(f"--- Result {i} (distance: {r['distance']:.4f}) ---")
    print(f"Source: {r['source']}")
    print(f"Text:   {r['text'][:200]}...")
    print()

## 3. RAG Agent

The agent combines retrieval + generation. It retrieves context from the knowledge base, injects it into the system prompt, and calls OpenAI.

### 3a. Initialize the agent

In [ ]:
from agent.rag_agent import RAGAgent

agent = RAGAgent(config=cfg, knowledge_base=kb)
print("Agent ready.")

### 3b. Ask a question

In [ ]:
answer = agent.ask("What is the return policy?")
print(answer)

### 3c. Multi-turn conversation

In [ ]:
answer2 = agent.ask("How long does the refund take?")
print(answer2)

In [ ]:
# View conversation history
for msg in agent.history:
    role = msg["role"].upper()
    content = msg["content"][:150]
    print(f"[{role}] {content}...")
    print()

### 3d. Reset and start over

In [ ]:
agent.reset_history()
print(f"History cleared. Length: {len(agent.history)}")

## 4. Custom System Message

You can override the system message at runtime by creating a new config.

In [ ]:
import os

# Override system message via env var
os.environ["SYSTEM_MESSAGE"] = (
    "You are a pirate AI assistant. Answer questions using the knowledge base "
    "but always talk like a pirate. Arrr!"
)

pirate_cfg = Config(env_only=True)
pirate_agent = RAGAgent(config=pirate_cfg, knowledge_base=kb)

answer = pirate_agent.ask("What is the return policy?")
print(answer)

# Clean up
del os.environ["SYSTEM_MESSAGE"]